In [ ]:
# Importing Cell

# For simplifying expressions
from sympy import symbols, Eq, simplify, And, Or, Not
from sympy.logic import simplify_logic

# For analyzing time of some things 
# (for now just simplifying time)
import time


In [87]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
x = model.NewBoolVar("x")

solver = cp_model.CpSolver()
solver.Solve(model)

print("x =", solver.Value(x))


x = 0


In [70]:
# Some Definitions of Symbols (0,1,AND,OR)
_false = '0'
_true = '1'
_and = "AND"
_or = "OR"

In [ ]:
# Books
#  Creating the Modelling


# B = [ 
#       [ [1] ] ,                          # b0
#       [ [0,1] ] ,                        # b1
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b2
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b3
#     ]

B = [ # good example for testing rereads
      [ [1,2,3,4],[1,2,3],[1] ] ,      # b0
      [ [5] ] ,                        # b1
      [ [0] ] ,                        # b2
      [ [5], [1,2,0] ] ,               # b3
    ]

n_of_unknown_words = 6 #max number in B - 1

n_of_books_to_read = 4 # At most the amount of books; max value t+1 can have.

n_of_books = len(B)

In [71]:
# sympy example use part 0

X = [symbols("X[0]"),symbols("X[1]"),symbols("X[2]")]

A,B,C,D,E,F,G = symbols("A B C D E F G")

expr = E | (A & E) | (B & E) | (C & True) | (D & (F & G))

# expr = expr.subs({A : 3})

print(simplify(expr))  # still returns (A & B) | (A & ~B)

C | E | (D & F & G)


In [ ]:
# Books
#  Creating the Modelling


# B = [ 
#       [ [1] ] ,                          # b0
#       [ [0,1] ] ,                        # b1
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b2
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b3
#     ]

B = [ # good example for testing rereads
      [ [1,2,3,4],[1,2,3],[1] ] ,      # b0
      [ [5] ] ,                        # b1
      [ [0] ] ,                        # b2
      [ [5], [1,2,0] ] ,               # b3
    ]

n_of_unknown_words = 6 #max number in B - 1

n_of_books_to_read = 4 # At most the amount of books; max value t+1 can have.
# This variable n_of_books_to_read shouldnt be here in leximizer. 
# Should come directly from the User, so before the leximizer

n_of_books = len(B)

In [72]:
def transform_x_to_p(s,p_to_x=0):
    # transforms a string with form "Eq(X[b],t)" to "Eq(P[t],b)" or back
    # if p_to_x is 0 or 1 respectively

    if "E" not in s: return s

    if p_to_x == 0:
        # Remove "Eq(X[" prefix and ")" suffix
        inner = s[len("Eq(X["):-1]   # gives: "i],j"

        # Split at "],"
        i, j = inner.split("],")

        # Build the new string
        return f"Eq(P[{j}],{i})"
    else:
        # Remove "Eq(P[" prefix and ")" suffix
        inner = s[len("Eq(P["):-1] # gives: "i],j"
        i, j = inner.split("],") # Split at "],"
        return f"Eq(X[{j}],{i})"

        

# print(transform_x_to_p("Eq(X[245353],5)",0))

In [73]:
def tokenize(expr):
    tokens = []
    i = 0
    n = len(expr)

    while i < n:
        c = expr[i]

        # skip spaces
        if c.isspace():
            i += 1
            continue

        # Standalone parenthesis
        if c == '(' or c == ')':
            tokens.append(c)
            i += 1
            continue

        # Identifiers / words (AND, OR, f, foo, x23...)
        if c.isalpha():
            start = i
            while i < n and (expr[i].isalnum() or expr[i] == '_'):
                i += 1
            name = expr[start:i]

            # If next char is "(" → function call
            if i < n and expr[i] == '(':
                depth = 0
                start2 = i
                while i < n:
                    if expr[i] == '(':
                        depth += 1
                    elif expr[i] == ')':
                        depth -= 1
                        if depth == 0:
                            i += 1
                            break
                    i += 1
                tokens.append(name + expr[start2:i])  # full: f(2,3)
            else:
                tokens.append(name)
            continue

        # Numbers
        if c.isdigit():
            start = i
            while i < n and expr[i].isdigit():
                i += 1
            tokens.append(expr[start:i])
            continue

        # Commas or other punctuation
        tokens.append(c)
        i += 1

    # Piecing together l, [, w, ], [, t, ] into l[w][t]
    tokens2 = []
    i = 0
    
    while i < len(tokens):
        # Check for the pattern: l [ x ] [ y ]
        if (tokens[i] == "l" or tokens[i] == "L") and i + 6 < len(tokens) :
            # Build the merged token
            merged = tokens[i] + "[" + tokens[i+2] + "][" + tokens[i+5] + "]"
            tokens2.append(merged)
            i += 7  # Skip everything used
        else:
            tokens2.append(tokens[i])
            i += 1

    return tokens2

In [74]:
def clean_vector_sympy2(v, t_max, w_max):
    L = [[[] for t in range(t_max+1)] for w in range(w_max)]
    P = [[] for t in range(t_max+1)]

    for t in range(t_max+1):
        for w in range(w_max):
            L[w][t] = symbols(f"L[{w}][{t}]")
        P[t] = symbols(f"P[{t}]")  
    

    # Making 0 into False and 1 into True
    # Replacing x with X, so we use X[] and not x[] by accident
    # Replacing l with L, so we use L[][] and not l[][] by accident
    
    # Vector for the translation of v to the sympy symbols
    v_for_sympy = []

    # Translating from our version to SymPys
    i = 0
    while i < len(v):
        if v[i] == _false: v_for_sympy.append('False'); i+=1
        elif v[i] == _true: v_for_sympy.append('True'); i+=1
        elif v[i] == _and: v_for_sympy.append('&'); i+=1
        elif v[i] == _or: v_for_sympy.append('|'); i+=1
        else: 
            # turns all x to X and l to L, then turns all X to P
            v_i_translation = v[i].translate(str.maketrans({"x": "X", "l": "L"}))
            v_i_translation = transform_x_to_p(v_i_translation,0)
            v_for_sympy.append(v_i_translation); i+=1

    # Putting all together in a string
    v_as_string = " ".join(v_for_sympy)

    # Creating actual SymPy expression
    expr = eval(v_as_string)

    # Simplify expression
    simplified_expr = simplify_logic(simplify(expr),form='dnf')

    # Convert back to the vector format
    simplified_v = convert_sympy_str_to_vector2(str(simplified_expr))

    return simplified_v

def convert_sympy_str_to_vector2(s):
    
    # its for the cases of "Eq(P[w], 0)" to turn into "Eq(P[w],0)"
    s2 = s.replace(", ", ",")
    # separate string into different tokens
    # ("(Eq(P[w],0))" -> ["(","Eq(P[w],0)",")"])
    v = tokenize(s2)

    # Vector for the translation of v from the sympy symbols
    v_from_sympy = []

    # Translating from SymPys version to ours
    i = 0
    while i < len(v):
        if v[i] == 'False': v_from_sympy.append(_false); i+=1
        elif v[i] == 'True': v_from_sympy.append(_true); i+=1
        elif v[i] == '&': v_from_sympy.append(_and); i+=1
        elif v[i] == '|': v_from_sympy.append(_or); i+=1
        else: 
            # turns all P to X, then X to x and L to l
            v_i_translation = transform_x_to_p(v[i],1)
            v_i_translation = v_i_translation.translate(str.maketrans({"X": "x", "L": "l"}))
            v_from_sympy.append(v_i_translation); i+=1

    return v_from_sympy

In [75]:
def clean_vector_sympy(v, t_max, w_max, b_max):
    L = [[[] for t in range(t_max+1)] for w in range(w_max)]
    X = [[] for b in range(b_max)]

    for t in range(t_max+1):
        for w in range(w_max):
            L[w][t] = symbols(f"L[{w}][{t}]")
    
    for b in range(b_max):
        X[b] = symbols(f"X[{b}]")
    

    # Making 0 into False and 1 into True
    # Replacing x with X, so we use X[] and not x[] by accident
    # Replacing l with L, so we use L[][] and not l[][] by accident
    
    # Vector for the translation of v to the sympy symbols    
    v_for_sympy = []

    # Translating from our version to SymPys
    i = 0
    while i < len(v):
        if v[i] == _false: v_for_sympy.append('False'); i+=1
        elif v[i] == _true: v_for_sympy.append('True'); i+=1
        elif v[i] == _and: v_for_sympy.append('&'); i+=1
        elif v[i] == _or: v_for_sympy.append('|'); i+=1
        else: 
            # turns all x to X and l to L
            v_i_translation = v[i].translate(str.maketrans({"x": "X", "l": "L"}))
            v_for_sympy.append(v_i_translation); i+=1


    # Putting all together in a string
    v_as_string = " ".join(v_for_sympy)

    # Creating actual SymPy expression
    expr = eval(v_as_string)

    # Simplify expression
    simplified_expr = simplify_logic(simplify(expr),form='dnf')

    # Convert back to the vector format
    simplified_v = convert_sympy_str_to_vector(str(simplified_expr))

    return simplified_v


def convert_sympy_str_to_vector(s):
    
    # its for the cases of "Eq(X[w], 0)" to turn into "Eq(X[w],0)"
    s2 = s.replace(", ", ",")
    # separate string into different tokens
    # ("(Eq(X[w],0))" -> ["(","Eq(X[w],0)",")"])
    v = tokenize(s2)

    # Vector for the translation of v from the sympy symbols
    v_from_sympy = []

    # Translating from SymPys version to ours
    i = 0
    while i < len(v):
        if v[i] == 'False': v_from_sympy.append(_false); i+=1
        elif v[i] == 'True': v_from_sympy.append(_true); i+=1
        elif v[i] == '&': v_from_sympy.append(_and); i+=1
        elif v[i] == '|': v_from_sympy.append(_or); i+=1
        else: 
            # turns all X to x and L to l
            v_i_translation = v[i].translate(str.maketrans({"X": "x", "L": "l"}))
            v_from_sympy.append(v_i_translation); i+=1

    return v_from_sympy

In [76]:
# Global Variables for program

v = [] # vector storing the ending expression

set_of_ks = {} # set that stores values like K(b,w,t,bv) and k(b,s_d,w,t)

v_of_vocab_sets = []

In [77]:
def add_to_set_of_ks(base_k,v_prime):
    # adds to set_of_ks with base_k as key and v_prime as value
    if base_k not in set_of_ks:
        set_of_ks[base_k] = v_prime
        return True # Indicates that was added
    return False # Indicates that wasnt added (was already there)

In [78]:
def iterate_B(B): # iterates through Bs terms, 
    # places values in set_of_ks,
    # TBD (use b_vocab) -> changes B to eliminate redundancy,
    # TBD () -> removes book b from B if no word to be learned (B[b] is an empty list)

    
    for b in range(len(B)):
        b_vocab = set()
        for s_d in range(len(B[b])):
            
            b_vocab.update(B[b][s_d])
            # Add all words in s_d to vocabulary

            if len(B[b][s_d]) == 1:

                w = B[b][s_d][0]
                base_K = f"K_reread({b},{w},0)"
                add_to_set_of_ks(base_K,[_true])
                # if s_d of book b has 1 word (w), 
                # we register that we can learn w by reading book b
        
        # Add set of book vocab to the global list
        v_of_vocab_sets.append(b_vocab)
                

In [79]:
def print_vector(vec): # prints vectors values with " " in between them
    for x in vec:
        print(x, end=" ")
    print()  # final newline

In [80]:
def vector_x_eq_int(b, t):
    # x_eq_int = ["(",f"Eq(x[{b}],{t})",")"]
    x_eq_int = [f"Eq(x[{b}],{t})"]
    return x_eq_int

def andify(vector_of_ands): # vector_of_ands is a vector of vectors
    v = []
    and_symbol_string = _and
    
    for i in range(len(vector_of_ands)):
        v.append("(")
        if vector_of_ands[i] == []:
            # if empty its the same as AND 1
            v.append('1')
        else:
            v.extend(vector_of_ands[i])
        v.append(")")
        if i != len(vector_of_ands)-1: 
            # if not the last, append an "AND" after
            v.append(and_symbol_string)
    return v

def orify(vector_of_ors): # vector_of_ors is a vector of vectors
    v = []
    or_symbol_string = _or
    
    for i in range(len(vector_of_ors)):
        v.append("(")
        if vector_of_ors[i] == []: 
            # if empty its the same as OR 0
            v.append(_false)
        else:
            v.extend(vector_of_ors[i])
        v.append(")")
        if i != len(vector_of_ors)-1: 
            # if not the last, append an "OR" after
            v.append(or_symbol_string)
    return v


In [81]:
def K_reread(b, w, t): # Can I learn word w from book b at time t?
    # This K is the older version, which allows rereads, 
    # but hopefully can provide more aggregation, leaving the reread to be dealt by another constraint
    
    # Check if we know K (we registered some values with t==0 in iterate_B)
    if t == 0:
        base_K = f"K_reread({b},{w},{t})"
        if base_K in set_of_ks:
            return set_of_ks[base_K]
    
    # check if book b has w
    if w not in v_of_vocab_sets[b]:
        # w is not to be found in book b

        if t == 0:
            # we read no books yet (t = 0) and 
            # book b doesnt have w in it

            # So, we cant learn it through book b, and return 0
            return [_false]
        
        # maybe we could have learned it in a past book
        # So, we check if we knew it at t-1 (l[w][t-1])

        # Check if they are True or False or has size 1 (is Eq(x[b],t)) 
        # so we can replace them already instead of a string of l[w][t-1]
        if ( l[w][t-1] == [_false] or l[w][t-1] == [_true] 
             or len(l[w][t-1]) == 1):
            return l[w][t-1]

        v_prime = [f"l[{w}][{t-1}]"]
        # v_prime = l[w][t-1]
        # ------ Change l ------

        return v_prime
    
        
    
    # At this point, we know:
    #   1. Word w is in b
    #   2. Word w is not alone in a sentence in b (else it would be in vocab)

    v_prime = []

    v_of_ors = []

    for s_d in range(len(B[b])):
        
        if w in B[b][s_d]:

            # This 'if' will never be entered because of "At this point, we know:" point 2
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                # So, we can return a 1, cause we will learn w in this book b
                return [_true]

            # if w is not alone, we need to know
            # all other words omega
            v_of_ands = []
            for omega in B[b][s_d]:

                if omega == w:
                    continue
                
                v_of_ands.append(k(b, s_d, omega, t))
            v_of_ands = andify(v_of_ands) 
            # Make the AND of all ks of the words in s_d

            v_of_ors.append(v_of_ands)
    v_prime = orify(v_of_ors)
    # Make the OR of knowing w in all sentences in book b

    return v_prime


In [82]:
def k(b, s_d, omega, t): # Do I know word omega in sentence s_d of b at time t?
    # return [f"k({b},{s_d},{omega},{t})"]
    base_k = f"k({b},{s_d},{omega},{t})"

    # Check if k(b,s_d,omega,t) was already calculated 
    # before and return its value if it was
    if base_k in set_of_ks:
        return set_of_ks[base_k]

    
    if len(B[b][s_d]) == 1 and B[b][s_d][0] == omega:
        # omega is the only word in sentence s_d of book b

        # Add before returning
        add_to_set_of_ks(base_k,[_true])

        # So, we learn omega reading book b at s_d. 
        # Hence, returning 1 for value of k
        return [_true]
    

    
    if s_d == 0: # starting sentence
        if t == 0: # starting book
            # if we are at the starting s_d and starting book
            # and s_d has more than 1 word, we cant learn it
            # so we return [0]

            # Add before returning
            add_to_set_of_ks(base_k,[_false])

            return [_false]
        
        
        # if first sentence, we can only look back at t-1
        
        # Check if they are True or False or has size 1 (is Eq(x[b*],t*) or l[w*][t*])
        # so we can replace them already instead of a string of l[omega][t-1]
        if ( l[omega][t-1] == [_false] or l[omega][t-1] == [_true] 
             or len(l[omega][t-1]) == 1):
            # if l[omega][t-1] is a single value, 
            # we can return it as the value to simplify later

            # Add before returning
            add_to_set_of_ks(base_k,l[omega][t-1])
            
            return l[omega][t-1]

        # Add before returning
        add_to_set_of_ks(base_k,[f"l[{omega}][{t-1}]"])
        # add_to_set_of_ks(base_k,l[omega][t-1])
        # --- Change l ---

        return [f"l[{omega}][{t-1}]"]
        # return l[omega][t-1]
        # --- Change l ---
    
    v_prime = []

    v_of_ors = []

    # EITHER we knew omega in the last sentence (s_d-1)
    v_of_ors.append(k(b, s_d-1, omega, t))

    # OR if (omega is in s_d) we know ALL the other words w 
    # in the same s_d of book b ( make an AND of k(b,s_d-1,w,t) )
    # If (omega not in s_d) we can just rely on knowing it in s_d-1, 
    # which is already in v_of_ors
    if omega in B[b][s_d]:
        v_of_ands = []

        for w in B[b][s_d]:
            if w == omega:
                continue

            v_of_ands.append(k(b, s_d-1, w, t))

        v_of_ands = andify(v_of_ands)
        v_of_ors.append(v_of_ands)
    
    v_prime = orify(v_of_ors)

    # Add before returning
    add_to_set_of_ks(base_k,v_prime)

    return v_prime

In [ ]:
# Adding Values to set_of_ks
iterate_B(B) # puts vaues in set_of_ks

In [84]:
l = [[[] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]
m = [[[] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

# For measuring total time for all expressions to be simplified
total_time = 0.0

for t in range(n_of_books_to_read):
    # verify run time
    vector_time_t = []
    for w in range(n_of_unknown_words):
        
        v_of_ors1 = []

        # Create "l[w][t-1] OR"
        if t != 0:
            if l[w][t-1] != [_false]:
                
                # Check if l[w][t-1] == [1], 
                # if yes we can say that any l[w][t*>t-1] == 1
                # So we just return [1]
                if l[w][t-1] == [_true]:
                    l[w][t] = [_true]
                    continue

                # Check if has size 1 (is Eq(x[b*],t*) or l[w*][t*])
                if (len(l[w][t-1]) == 1):
                    v_of_ors1.append(l[w][t-1])

                # If not, just add string version
                else:    
                    v_of_ors1.append([f"l[{w}][{t-1}]"])
                    # v_of_ors1.append(l[w][t-1])
                    # --- Change l ---

        
        v_prime = []
        v_of_ors2 = []

        for b in range(n_of_books):

            v_of_ands = []

            # Create "(x[b] == t) AND"
            v_of_ands.append(vector_x_eq_int(b,t))
            # Create "K_reread(b,w,t)""
            v_of_ands.append(K_reread(b,w,t))

            v_of_ands = andify(v_of_ands)
            v_of_ors2.append(v_of_ands)
        
        v_of_ors2 = orify(v_of_ors2)
        v_of_ors1.append(v_of_ors2)

        v_prime = orify(v_of_ors1)

        l[w][t] = v_prime

        start = time.perf_counter() # verify run time

        # Use SymPy to simplify ("clean") expressions

        # This cleans with the perspective of Eq(P[t],b), 
        # simplifying SOME (not all) cases of reading two books at once
        l[w][t] = clean_vector_sympy2(l[w][t],t,n_of_unknown_words)

        # This cleans with the perspective of Eq(X[b],t), 
        # simplifying SOME (not all) cases of rereading
        l[w][t] = clean_vector_sympy(l[w][t],t,n_of_unknown_words,n_of_books)

        end = time.perf_counter() # verify run time


        # Creating m[*][*] to indicate if l[w][t]
        # should be a variable or not
        if len(l[w][t]) != 1 or t == n_of_books_to_read-1:
            m[w][t] = True
        else: m[w][t] = False


        # verify run time
        elapsed = end - start
        total_time += elapsed
        vector_time_t.append(f"{elapsed:.5f}")
    # print(f"l[*][{t}] time = {vector_time_t}") # verify run time
print(f"Total time: {round(total_time,5)}") # verify run time


Total time: 1.5195


In [85]:
# Results Display
for t in range(n_of_books_to_read): # n_of_books_to_read
    for w in range(n_of_unknown_words): # n_of_unknown_words
        print(f"l[{w}][{t}]:")
        print("     ",end="")
        print_vector(l[w][t])
        
        print()

# print_vector(l[3][1])

l[0][0]:
     Eq(x[2],0) 

l[1][0]:
     Eq(x[0],0) 

l[2][0]:
     0 

l[3][0]:
     0 

l[4][0]:
     0 

l[5][0]:
     Eq(x[1],0) OR Eq(x[3],0) 

l[0][1]:
     Eq(x[2],0) OR Eq(x[2],1) 

l[1][1]:
     Eq(x[0],0) OR Eq(x[0],1) 

l[2][1]:
     0 

l[3][1]:
     0 

l[4][1]:
     0 

l[5][1]:
     l[5][0] OR Eq(x[1],1) OR Eq(x[3],1) 

l[0][2]:
     l[0][1] OR Eq(x[2],2) 

l[1][2]:
     l[1][1] OR Eq(x[0],2) 

l[2][2]:
     l[0][1] AND l[1][1] AND Eq(x[3],2) 

l[3][2]:
     0 

l[4][2]:
     0 

l[5][2]:
     l[5][1] OR Eq(x[1],2) OR Eq(x[3],2) 

l[0][3]:
     l[0][2] OR Eq(x[2],3) OR ( l[1][2] AND l[2][2] AND Eq(x[3],3) ) 

l[1][3]:
     l[1][2] OR Eq(x[0],3) OR ( l[0][2] AND l[2][2] AND Eq(x[3],3) ) 

l[2][3]:
     l[2][2] OR ( l[0][2] AND l[1][2] AND Eq(x[3],3) ) 

l[3][3]:
     l[1][2] AND l[2][2] AND Eq(x[0],3) 

l[4][3]:
     0 

l[5][3]:
     l[5][2] OR Eq(x[1],3) OR Eq(x[3],3) 



In [86]:
# Rascunho em Python 5 - Time for SymPy Simplifier

l[*][0] time = ['0.00147', '0.00124', '0.00053', '0.00042', '0.00046', '0.00526']
l[*][1] time = ['0.21801', '0.22605', '0.05498', '0.00146', '0.00071', '0.04474']
l[*][2] time = ['0.10026', '0.09758', '0.12758', '0.39025', '0.08802', '0.04537']
l[*][3] time = ['0.09954', '0.09630', '0.11989', '0.17735', '0.16393', '0.04442']

Total time: 2.10584

# -----------------------------------------------------------------------

l[*][0] time = ['0.00144', '0.00109', '0.00047', '0.00037', '0.00034', '0.00479']
l[*][1] time = ['0.04212', '0.04086', '0.06128', '0.00069', '0.00047', '0.04188']
l[*][2] time = ['0.10211', '0.09477', '0.08100', '0.01761', '0.00065', '0.04383']
l[*][3] time = ['0.09419', '0.09151', '0.25212', '0.06179', '0.02016', '0.04473']

Total time: 1.10027

SyntaxError: invalid syntax (2729774983.py, line 3)

In [ ]:
# Rascunho em Python 4 - Expressions

# l[0][0]:
#      Eq(x[2],0) 

# l[1][0]:
#      Eq(x[0],0) 

# l[2][0]:
#      0 

# l[3][0]:
#      0 

# l[4][0]:
#      0 

# l[5][0]:
#      Eq(x[1],0) OR Eq(x[3],0) 

# l[0][1]:
#      Eq(x[2],0) OR Eq(x[2],1) 

# l[1][1]:
#      Eq(x[0],0) OR Eq(x[0],1) 

# l[2][1]:
#      0 

# l[3][1]:
#      0 

# l[4][1]:
#      0 

# l[5][1]:
#      l[5][0] OR Eq(x[1],1) OR Eq(x[3],1) 

# l[0][2]:
#      l[0][1] OR Eq(x[2],2) 

# l[1][2]:
#      l[1][1] OR Eq(x[0],2) 

# l[2][2]:
#      l[0][1] AND l[1][1] AND Eq(x[3],2) 

# l[3][2]:
#      0 

# l[4][2]:
#      0 

# l[5][2]:
#      l[5][1] OR Eq(x[1],2) OR Eq(x[3],2) 

# l[0][3]:
#      l[0][2] OR Eq(x[2],3) OR ( l[1][2] AND l[2][2] AND Eq(x[3],3) ) 

# l[1][3]:
#      l[1][2] OR Eq(x[0],3) OR ( l[0][2] AND l[2][2] AND Eq(x[3],3) ) 

# l[2][3]:
#      l[2][2] OR ( l[0][2] AND l[1][2] AND Eq(x[3],3) ) 

# l[3][3]:
#      l[1][2] AND l[2][2] AND Eq(x[0],3) 

# l[4][3]:
#      0 

# l[5][3]:
#      l[5][2] OR Eq(x[1],3) OR Eq(x[3],3)

SyntaxError: invalid syntax (1110901855.py, line 3)